 # QQQ数据分析

本笔记分析QQQ的数据以回答以下问题：
1. QQQ当前的估值是否合理？
2. QQQ的中短期趋势如何？

In [79]:
#imports

import yfinance as yf
import pandas as pd
import datetime as dt

pd.options.display.float_format = '{:,.2f}'.format

In [61]:
# get data

# get a list of QQQ member

#获取纳斯达克100成分
table=pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')
df_holding = table[3]
df_holding.to_csv("data\share\nasdaq100.csv")


In [ ]:
def get_stock_info(ticker):
    stock_info={}
    df=yf.download(ticker,start=dt.date.today()-dt.timedelta(days=366),auto_adjust=True,progress=False)
    stock_info=yf.Ticker(ticker).info
    stock_info['1w_chg_pct']=100*(df.loc[max(df.index)-dt.timedelta(weeks=1):,'Close'][-1]/df.loc[max(df.index)-dt.timedelta(weeks=1):,'Close'][0]-1)
    stock_info['1m_chg_pct']=100*(df.loc[max(df.index)-dt.timedelta(days=30):,'Close'][-1]/df.loc[max(df.index)-dt.timedelta(days=30):,'Close'][0]-1)
    stock_info['1y_chg_pct']=100*(df.loc[max(df.index)-dt.timedelta(days=365):,'Close'][-1]/df.loc[max(df.index)-dt.timedelta(days=365):,'Close'][0]-1)
    stock_info['max_drawdown']=100*(df.Close[-1]/max(df.Close)-1)
    stock_info['ticker']=ticker
    return stock_info

df_stock=pd.DataFrame()
for ticker in df_holding.Ticker:
    try:
        stock_info=get_stock_info(ticker)
        df_stock=df_stock.append(stock_info,ignore_index=True)
    except:
        pass

df_stock['ratecode']=df_stock['financialCurrency']+'-'+df_stock['currency']
df_stock['ratecode']
df_stock['USD-USD']=1
df_stock['CNY-USD']=0.16
df_stock['TWD-USD']=0.036
df_stock['CNY-HKD']=1.22
df_stock['EUR-USD']=1.13
df_stock['rate']=df_stock['USD-USD'].where(df_stock['ratecode']=='USD-USD')
df_stock.loc[df_stock['rate'].isna(),['rate']]=df_stock['CNY-USD'].where(df_stock['ratecode']=='CNY-USD')
df_stock.loc[df_stock['rate'].isna(),['rate']]=df_stock['TWD-USD'].where(df_stock['ratecode']=='TWD-USD')
df_stock.loc[df_stock['rate'].isna(),['rate']]=df_stock['EUR-USD'].where(df_stock['ratecode']=='EUR-USD')
df_stock.loc[df_stock['rate'].isna(),['rate']]=df_stock['CNY-HKD'].where(df_stock['ratecode']=='CNY-HKD')
#df_stock[['ticker','currency','financialCurrency','rate']]

In [6]:
df_stock.to_csv('../data/share/qqq_data.csv')

In [37]:
df_stock.drop(df_stock[df_stock['ticker']=='GOOGL'].index,inplace=True)

## QQQ 估值指标监测

In [69]:
df_metric=pd.DataFrame()
df_metric['ticker']=df_stock['ticker']
df_metric['market_cap']=df_stock['marketCap']
df_metric['revenue']=df_stock['totalRevenue']*df_stock['rate']
df_metric['revenue_growth']=df_stock['revenueGrowth']
df_metric['cashflow']=df_stock['operatingCashflow']*df_stock['rate']
df_metric['net_income']=df_stock['netIncomeToCommon']*df_stock['rate']
df_metric['gross_profit']=df_stock['grossProfits']*df_stock['rate']
total_market_cap=df_stock['marketCap'].sum()
df_metric['weight']=df_metric['market_cap']/total_market_cap

In [96]:
etf_stat=dict()
etf_stat['market_cap']=(df_metric['market_cap']*df_metric['weight']).sum()
etf_stat['revenue']=(df_metric['revenue']*df_metric['weight']).sum()
etf_stat['revenue_growth']=((df_metric['revenue']-df_metric['revenue']/(1+df_metric['revenue_growth']))*df_metric['weight']).sum()/etf_stat['revenue']
etf_stat['ps']=etf_stat['market_cap']/etf_stat['revenue']
etf_stat['gross_profit']=(df_metric['gross_profit']*df_metric['weight']).sum()
etf_stat['pgp']=etf_stat['market_cap']/etf_stat['gross_profit']
etf_stat['gross_margin']=100*etf_stat['gross_profit']/etf_stat['revenue']
etf_stat['cashflow']=(df_metric['cashflow']*df_metric['weight']).sum()
etf_stat['pcf']=etf_stat['market_cap']/etf_stat['cashflow']
etf_stat['cf_rate']=100/etf_stat['pcf']
etf_stat['cf_margin']=100*etf_stat['cashflow']/etf_stat['revenue']
etf_stat['net_income']=(df_metric['net_income']*df_metric['weight']).sum()
etf_stat['pe']=etf_stat['market_cap']/etf_stat['net_income']
etf_stat['income_rate']=100/etf_stat['pe']
etf_stat['profit_margin']=100*etf_stat['net_income']/etf_stat['revenue']



In [97]:
etf={key:etf_stat[key] for key in ['revenue_growth','ps','pgp','gross_margin','pcf','cf_rate','cf_margin','pe','income_rate','profit_margin']}

In [98]:
df_etf_stat=pd.DataFrame.from_dict(etf,orient='index',columns=['QQQ'])

In [99]:
df_etf_stat

,QQQ
revenue_growth,0.22
ps,7.87
pgp,18.11
gross_margin,43.48
pcf,27.60
cf_rate,3.62
cf_margin,28.54
pe,34.79
income_rate,2.87
profit_margin,22.64
